In [161]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, FloatType, DateType
import datetime, time

print(sc)

In [135]:
spark = (SparkSession
         .builder
         .appName("Catch you merchant")
         .config("spark.sql.warehouse.dir", "/opt/jupyter_workspace/spark-warehouse")
         .getOrCreate())
print(spark)

In [165]:
workspace = "/Users/AUM/Desktop/MAPS/"

df = (spark
     .read
     .option("header", "true")
     .option("inferSchema", "true")
     .csv(workspace + "Workbook2.csv"))

In [166]:
df = df.dropna()

df.show()

+-----+-----------+-------+---------+---------+------+---------+------+------+--------+
|ar_id|fm_to_ar_id|txn_amt|svc_br_no|opm_tp_cd|txn_cd|ptn_yyyyy|ptn_mm|ptn_dd|  txn_tm|
+-----+-----------+-------+---------+---------+------+---------+------+------+--------+
|  11a|        13c|    100|      900|       CR|     0|     2016|     8|     3| 9:58:18|
|  11a|        13c|1000000|      900|       DR|     0|     2016|     8|     5|10:58:18|
|  11a|        13c|  10000|      900|       DR|     0|     2016|     9|    10|11:58:18|
|  11a|        13c|  10000|      900|       DR|     0|     2016|     9|    10|11:58:18|
|  11a|        14d| 200000|      901|       DR|     0|     2016|    10|     3|12:58:18|
|  12b|        15c|  30000|      902|       DR|     0|     2016|    12|     3|13:58:18|
|  12b|        16d| 200000|      903|       CR|     0|     2016|     8|    30|14:58:18|
|  12b|        16d|   5000|      904|       CR|     0|     2016|     8|    13|15:58:18|
+-----+-----------+-------+-----

In [181]:
date = udf(lambda y, m, d : datetime.datetime(y, m ,d), DateType())

day_of_week = udf(lambda date : date.weekday(), IntegerType())

time = udf(lambda time_string : time_string[:-6], IntegerType())

NameError: name 'InteherType' is not defined

In [179]:
df = df.withColumn("date", date(df["ptn_yyyyy"], df["ptn_mm"], df["ptn_dd"]))

df= df.withColumn("day_of_week", day_of_week(df["date"]))

df = df.withColumn("time", time(df["txn_tm"]))

In [180]:
df.show()

+-----+-----------+-------+---------+---------+------+---------+------+------+--------+----------+-----------+----------+
|ar_id|fm_to_ar_id|txn_amt|svc_br_no|opm_tp_cd|txn_cd|ptn_yyyyy|ptn_mm|ptn_dd|  txn_tm|      date|day_of_week|      time|
+-----+-----------+-------+---------+---------+------+---------+------+------+--------+----------+-----------+----------+
|  11a|        13c|    100|      900|       CR|     0|     2016|     8|     3| 9:58:18|2016-08-03|          2|1900-01-01|
|  11a|        13c|1000000|      900|       DR|     0|     2016|     8|     5|10:58:18|2016-08-05|          4|1900-01-01|
|  11a|        13c|  10000|      900|       DR|     0|     2016|     9|    10|11:58:18|2016-09-10|          5|1900-01-01|
|  11a|        13c|  10000|      900|       DR|     0|     2016|     9|    10|11:58:18|2016-09-10|          5|1900-01-01|
|  11a|        14d| 200000|      901|       DR|     0|     2016|    10|     3|12:58:18|2016-10-03|          0|1900-01-01|
|  12b|        15c|  300

In [138]:
data = df.select("ar_id").distinct().union(df.select("fm_to_ar_id").distinct()).dropna()
data.show()

+-----+
|ar_id|
+-----+
|  11a|
|  12b|
|  16d|
|  15c|
|  14d|
|  13c|
+-----+



In [ ]:
# define variable
month = 10

In [139]:
# define functions

def noDepositBranchVisit():
    return df.select("ar_id", "svc_br_no").groupby("ar_id").agg({"svc_br_no" : "count"})

def noDepositBranchVisitUnique():
    return df.select("ar_id", "svc_br_no").distinct().groupby("ar_id").agg({"svc_br_no" : "count"})

def noDepositTransferIn():
    return 

def noDepositTransfrtOut():
    return

def ratioDepositBranchVisit(visit, unique_visit):
    return visit / unique_visit

In [140]:
noDepositBranchVisit().show()

noDepositBranchVisitUnique().show()

+-----+----------------+
|ar_id|count(svc_br_no)|
+-----+----------------+
|  11a|               5|
|  12b|               3|
+-----+----------------+

+-----+----------------+
|ar_id|count(svc_br_no)|
+-----+----------------+
|  11a|               2|
|  12b|               3|
+-----+----------------+

